我们分析了每个lambda下预测成功的embedding的分布情况，显示不同lambda下embedding的分布几乎一样

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
from tqdm import tqdm

from config import train_config as config
import utils
import data_utils.mtop_loader as dataloader
from models.embedding_model import EmbeddingModel
from models.policy_network import RBFPolicyNetwork

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

MODEL_PATH =  "cache/lambda_icl_qwen_0.6b/1210_2023_best.pt"

def check_model_distribution():

    device = utils.device
    logger.info(f"Using device: {device}")

    # 2. 初始化 Embedding Model (必须与 main.py 一致，用于编码 query)
    logger.info(f"Loading EmbeddingModel: {config.EMBEDDING_MODEL_NAME} ...")
    embedding_model = EmbeddingModel(model_name=config.EMBEDDING_MODEL_NAME)

    # 3. 初始化 Policy Network (Agent)
    logger.info("Initializing RBFPolicyNetwork (num_centers=1024)...")
    agent = RBFPolicyNetwork(
        embedding_dim=embedding_model.dim,
        num_centers=1024,  
        dropout=0.0       
    ).to(device)

    # 4. 加载模型权重
    if os.path.exists(MODEL_PATH):
        logger.info(f"Loading checkpoint from: {MODEL_PATH}")
        try:
            state_dict = torch.load(MODEL_PATH, map_location=device)

            if isinstance(state_dict, dict) and 'model_state_dict' in state_dict:
                agent.load_state_dict(state_dict['model_state_dict'])
            else:
                agent.load_state_dict(state_dict)
            logger.info("Model loaded successfully.")
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            return
    else:
        logger.error(f"Model file not found: {MODEL_PATH}")
        return

    agent.eval()

    # 5. 获取数据 (使用 dev 集)
    # 模拟 main.py 中的 val_loader
    logger.info("Loading Validation Data (split='dev')...")
    val_loader = dataloader.get_dataloader(
        split='dev', 
        batch_size=64, 
        shuffle=False
    )

    # 6. 推理
    all_actions = []
    all_probs = []
    
    logger.info("Running Inference...")
    
    # 只取第一个 batch 进行详细分析
    batch_data = next(iter(val_loader))
    
    # main.py 的 dataloader 返回的是 List[Dict]
    query_texts = [item['query'] for item in batch_data]
    
    with torch.no_grad():
        # 实时编码 (同 main.py)
        query_embs = embedding_model.encode(query_texts) # Tensor on device
        
        # 获取模型输出
        logits, values = agent.get_logits_and_values(query_embs)
        probs = F.softmax(logits, dim=-1)
        actions = torch.argmax(probs, dim=-1)
        
        all_actions = actions.cpu().numpy()
        all_probs = probs.cpu().numpy()

    # 7. 绘图与分析
    plot_analysis(all_actions, all_probs)

def plot_analysis(actions, probs):
    """
    绘制详细的分布图
    """
    num_samples = len(actions)
    # 假设 Action 0-20 对应 Lambda 0.0 - 1.0 (step 0.05)
    lambda_values = np.arange(21) * 0.05
    
    plt.figure(figsize=(15, 6))

    # --- 图 1: 最终选择的 Lambda 计数 ---
    plt.subplot(1, 2, 1)
    # 统计每个 Lambda 被选中的次数
    counts = np.bincount(actions, minlength=21)
    
    plt.bar(lambda_values, counts, width=0.04, color='teal', alpha=0.8, edgecolor='black')
    plt.title(f'Selected Lambda Distribution (Batch Size={num_samples})')
    plt.xlabel('Lambda Value')
    plt.ylabel('Count (Frequency)')
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.grid(axis='y', alpha=0.3)

    # --- 图 2: 平均概率分布 (模型是否犹豫) ---
    plt.subplot(1, 2, 2)
    mean_probs = np.mean(probs, axis=0)
    
    plt.bar(lambda_values, mean_probs, width=0.04, color='skyblue', edgecolor='blue', alpha=0.8)
    plt.title('Average Predicted Probability (Model Confidence)')
    plt.xlabel('Lambda Value')
    plt.ylabel('Avg Probability')
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.ylim(0, 1.0)
    plt.grid(axis='y', alpha=0.3)

    # 计算平均熵
    entropy = -np.sum(probs * np.log(probs + 1e-9), axis=1).mean()
    plt.text(0.05, 0.9, f"Avg Entropy: {entropy:.4f}", transform=plt.gca().transAxes, 
             fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.show()

    # --- 打印文字报告 ---
    print("\n" + "="*40)
    print(f"Analysis Report (Samples: {num_samples})")
    print("="*40)
    print(f"Avg Entropy: {entropy:.4f} (Max possible: {np.log(21):.4f})")
    
    print("\nTop 5 Preferred Lambdas (by selection count):")
    top_indices = np.argsort(counts)[::-1][:5]
    for idx in top_indices:
        if counts[idx] > 0:
            print(f"  Lambda {idx*0.05:.2f}: {counts[idx]} samples")
            
    print("\nTop 5 Highest Probability Mass (on average):")
    top_prob_indices = np.argsort(mean_probs)[::-1][:5]
    for idx in top_prob_indices:
        print(f"  Lambda {idx*0.05:.2f}: {mean_probs[idx]:.4f}")
    
    # 检查是否是“平顶山”
    std_probs = np.std(mean_probs)
    if std_probs < 0.02:
        print("\n[WARNING] Probability distribution is very flat! The model fails to distinguish between actions.")

if __name__ == "__main__":
    check_model_distribution()

我们继续分析成功预测和失败预测 embedding的分布情况

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from config import train_config as config
import utils
import data_utils.mtop_loader as dataloader
from models.embedding_model import EmbeddingModel
from models.llm_wrapper import LLMWrapper
from models.policy_network import RBFPolicyNetwork
from engine.sampler import EpisodeSampler
from engine.reward_computer import RewardComputer

MODEL_PATH = "cache/lambda_icl_qwen_0.6b/1210_2023_best.pt" 
CHECK_BATCHES = 10  # 检查多少个 Batch
BATCH_SIZE = 16     # 每个 Batch 的大小

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def check_advantage_stats():
    device = utils.device
    
    logger.info("Initializing models...")
    emb_model = EmbeddingModel(model_name=config.EMBEDDING_MODEL_NAME)
    llm_wrapper = LLMWrapper(model_name=config.LLM_MODEL_NAME)
    
    agent = RBFPolicyNetwork(
        embedding_dim=emb_model.dim,
        num_centers=1024, 
        dropout=0.0
    ).to(device)
    
    if os.path.exists(MODEL_PATH):
        logger.info(f"Loading checkpoint: {MODEL_PATH}")
        state_dict = torch.load(MODEL_PATH, map_location=device)
        if isinstance(state_dict, dict) and 'model_state_dict' in state_dict:
            agent.load_state_dict(state_dict['model_state_dict'])
        else:
            agent.load_state_dict(state_dict)
    else:
        logger.warning(f"Checkpoint not found at {MODEL_PATH}, using random init!")

    agent.eval()
    
    corpus_data, corpus_embeddings = dataloader.get_corpus()
    train_loader = dataloader.get_dataloader(
        split='train', batch_size=BATCH_SIZE, shuffle=True, nums=256
    )
    
    sampler = EpisodeSampler(
        policy_network=agent, 
        embedding_model=emb_model, 
        num_examples=config.NUM_EXAMPLES
    )
    
    reward_computer = RewardComputer(
        gamma=config.REWARD_GAMMA, 
        lambda_=config.REWARD_LAMBDA, 
        system_prompt=config.SYSTEM_PROMPT
    )
    
    all_advantages = []
    all_rewards = []
    all_values = []
    
    logger.info(f"Collecting {CHECK_BATCHES} batches to analyze Advantage distribution...")
    
    iterator = iter(train_loader)
    for i in range(CHECK_BATCHES):
        try:
            batch_data = next(iterator)
        except StopIteration:
            break
            
       
        buffer = sampler.collect_episodes(
            query_batch=batch_data,
            corpus=corpus_data,
            corpus_embeddings=corpus_embeddings
        )
        
        # B. 计算 Advantage (Compute)
        # 这步很关键，它会调用 Critic 计算 Value，并计算 GAE Advantage
        # 此时得到的 buffer.advantages 是原始的 (Raw)，还没被 Normalize
        buffer = reward_computer.compute_rewards_and_advantages(
            buffer=buffer,
            llm_wrapper=llm_wrapper,
            check_correct_fn=dataloader.check_correct,
        )
        
        # 收集数据 (只取第一个 step 的，如果是单步任务)
        # buffer.advantages shape: [Batch, Steps]
        advs = buffer.advantages.flatten().cpu().numpy()
        rews = buffer.rewards.flatten().cpu().numpy()
        vals = buffer.values.flatten().cpu().detach().numpy()
        
        all_advantages.extend(advs)
        all_rewards.extend(rews)
        all_values.extend(vals)
        
        print(f"Batch {i+1}/{CHECK_BATCHES} processed.")

    # 4. 统计与可视化
    all_advantages = np.array(all_advantages)
    all_rewards = np.array(all_rewards)
    all_values = np.array(all_values)
    
    # 阈值：绝对值小于 1e-4 视为 0
    zero_mask = np.abs(all_advantages) < 1e-4
    zero_ratio = np.sum(zero_mask) / len(all_advantages)
    
    print("\n" + "="*40)
    print("ADVANTAGE DISTRIBUTION ANALYSIS")
    print("="*40)
    print(f"Total Samples: {len(all_advantages)}")
    print(f"Zero Advantage Ratio (Abs < 1e-4): {zero_ratio:.2%}  <-- 重点关注这个")
    print(f"Mean Advantage: {np.mean(all_advantages):.4f}")
    print(f"Std Advantage:  {np.std(all_advantages):.4f}")
    print("-" * 20)
    print("Reward vs Value (Critic Accuracy):")
    print(f"Mean Reward: {np.mean(all_rewards):.4f}")
    print(f"Mean Value:  {np.mean(all_values):.4f}")
    
    # 简单分析 Critic 是否已经学会了预测“死数据”
    # 如果 Reward=1 且 Value≈1，或者 Reward=0 且 Value≈0，Advantage 就会是 0
    accurate_prediction_mask = np.abs(all_rewards - all_values) < 0.1
    print(f"Accurate Critic Prediction Ratio (|R-V| < 0.1): {np.sum(accurate_prediction_mask)/len(all_values):.2%}")

    # 绘制直方图
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    sns.histplot(all_advantages, bins=50, kde=False, color='purple')
    plt.title('Raw Advantage Distribution')
    plt.xlabel('Advantage value')
    plt.ylabel('Count')
    
    plt.subplot(1, 2, 2)
    plt.scatter(all_values, all_rewards, alpha=0.3, s=10)
    plt.title('Critic Value vs. Real Reward')
    plt.xlabel('Predicted Value (Critic)')
    plt.ylabel('Real Reward')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    check_advantage_stats()

INFO:__main__:Initializing models...
INFO:root:[EmbeddingModel] Loading embedding model: all-MiniLM-L6-v2...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:root:[EmbeddingModel] Model loaded. Embedding dimension: 384
INFO:models.llm_wrapper:Loading LLM environment: Qwen/Qwen3-0.6B...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
INFO:models.llm_wrapper:LLM 'Qwen/Qwen3-0.6B' loaded successfully.
INFO:models.policy_network:Initializing RBFPolicyNetwork: Input=384, Centers=1024 (RBF Layer)
INFO:__main__:Loading checkpoint: cache/lambda_icl_qwen_0.6b/1210_2023_best.pt
INFO:data_utils.mtop_loader:--- Creating Corpus (Action Space) ---
INFO:data_utils.mtop_loader:Loading full 'train' split (no global cache used)...
INFO:data_utils.mtop_loader:Loading EmbeddingModel

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.35 GiB. GPU 0 has a total capacity of 47.38 GiB of which 4.65 GiB is free. Including non-PyTorch memory, this process has 42.71 GiB memory in use. Of the allocated memory 39.58 GiB is allocated by PyTorch, and 2.73 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
import numpy as np
import os
from tqdm import tqdm
import re
import pandas as pd

from config import train_config as config
import utils
import data_utils.mtop_loader as dataloader
from models.embedding_model import EmbeddingModel
from models.llm_wrapper import LLMWrapper
from utils import device

NUM_SAMPLES_TO_INSPECT =128
OUTPUT_DIR = "results/analysis"
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "inspection_report.csv")

LAMBDA_CANDIDATES = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

utils.setup_logging(log_level="INFO")
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Starting CSV Inspection. Output: {OUTPUT_FILE}")

print("Loading Models...")
embedding_model = EmbeddingModel(config.EMBEDDING_MODEL_NAME)
llm_wrapper = LLMWrapper(config.LLM_MODEL_NAME)

corpus_data, corpus_embeddings_cpu = dataloader.get_corpus()
corpus_embeddings = corpus_embeddings_cpu.to(device)

train_raw, _ = dataloader.get_train_val_split_data(
    split='train',
    train_nums=config.PRETRAIN_NUMS,
    val_nums=64,
    seed=config.PRETRAIN_SEED
)

inspect_data = train_raw[:NUM_SAMPLES_TO_INSPECT] if NUM_SAMPLES_TO_INSPECT else train_raw
print(f"Inspecting {len(inspect_data)} samples...")

rows = []

for idx, sample in enumerate(tqdm(inspect_data, desc="Inspecting")):
    query_text = sample['query']
    target_answer = sample['answer']
    
    row_data = {
        "Index": idx,
        "Question": query_text,
        "Answer": target_answer,
    }
    
    query_emb = embedding_model.encode([query_text]) # (1, D)
    query_index = sample.get('corpus_index', -1)
    
    sim_scores_base = torch.matmul(query_emb, corpus_embeddings.T) # (1, Corpus)
    
    correct_lambdas = []

    for lam_val in LAMBDA_CANDIDATES:
        lambda_tensor = torch.tensor(lam_val, device=device)

        selected_indices = []
        selected_embs = torch.zeros((1, 0, embedding_model.dim), device=device)
        
        curr_mask = torch.zeros_like(sim_scores_base, dtype=torch.bool)
        if query_index >= 0:
            curr_mask[0, query_index] = True
        
        for t in range(config.NUM_EXAMPLES):
            if t == 0:
                step_scores = sim_scores_base.clone()
            else:
                sim_to_selected = torch.matmul(selected_embs, corpus_embeddings.T)
                diversity_penalty, _ = torch.max(sim_to_selected, dim=1)
                step_scores = (lambda_tensor * sim_scores_base) - ((1 - lambda_tensor) * diversity_penalty)
            
            step_scores.masked_fill_(curr_mask, -float('inf'))
            best_idx = torch.argmax(step_scores, dim=1).item()
            selected_indices.append(best_idx)
            curr_mask[0, best_idx] = True
            
            new_emb = corpus_embeddings[best_idx].unsqueeze(0).unsqueeze(0)
            selected_embs = torch.cat([selected_embs, new_emb], dim=1)

        examples = [corpus_data[i] for i in selected_indices]

        prompt = llm_wrapper.build_chat_prompt(config.SYSTEM_PROMPT, examples, query_text)
        pred_texts, _ = llm_wrapper.generate_for_evaluation([prompt], max_new_tokens=config.MAX_GEN_TOKENS)
        pred_text = pred_texts[0]

        is_correct = dataloader.check_correct(target_answer, pred_text)
        if is_correct:
            correct_lambdas.append(lam_val)

        mark = "✅" if is_correct else "❌"
        row_data[f"Pred_{lam_val}"] = f"{mark} {pred_text}"
        row_data[f"Retrieved_{lam_val}"] = ' '.join([str(ex) for ex in examples])

    row_data["Is_Solvable"] = len(correct_lambdas) > 0
    row_data["Best_Lambdas"] = str(correct_lambdas)
    
    rows.append(row_data)

df = pd.DataFrame(rows)

base_cols = ["Index", "Is_Solvable", "Best_Lambdas", "Question", "Answer"]

lambda_cols = []
for lam in LAMBDA_CANDIDATES:
    lambda_cols.append(f"Pred_{lam}")
    lambda_cols.append(f"Retrieved_{lam}")

df = df[base_cols + lambda_cols]

df = df.sort_values(by=["Is_Solvable", "Index"], ascending=[False, True])

df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig') # utf-8-sig 兼容 Excel 打开中文
print(f"Successfully saved inspection report to {OUTPUT_FILE}")
print(f"Solvable Samples: {df['Is_Solvable'].sum()}/{len(df)}")


In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="sk-49da8b1a6c6d4f3e8e7a74860f2d11f1", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

messages = [
    {"role": "system", "content": "You are an expert assistant for semantic parsing. Given a user utterance, you must convert it into its logical form representation."},
    {"role": "user", "content": "stop and delete the timer"}, {"role": "assistant", "content": "[IN:PAUSE_TIMER [SL:METHOD_TIMER timer ] ]"},
    {"role": "user", "content": "close out of timer"}, {"role": "assistant", "content": "[IN:PAUSE_TIMER  [SL:METHOD_TIMER timer ] ]"},
    {"role": "user", "content": "timer re-start"}, {"role": "assistant", "content": "[IN:RESUME_TIMER [SL:METHOD_TIMER timer ] ]"},
    {"role": "user", "content": "alter timer"}, {"role": "assistant", "content": "[IN:UPDATE_TIMER [SL:METHOD_TIMER  timer ] ]"},
    {"role": "user", "content": "Cancel timer"}, {"role": "assistant", "content": "[IN:DELETE_TIMER [SL:METHOD_TIMER timer ] ]"},
    # {"role": "user", "content": "timer stop"}, {"role": "assistant", "content": "[IN:PAUSE_TIMER [SL:METHOD_TIMER timer ]  ]"},
    # {"role": "user", "content": "stop and remove current timer"}, {"role": "assistant", "content": "[IN:PAUSE_TIMER [SL:METHOD_TIMER timer ] ]"},
    # {"role": "user", "content": "turn off timer"}, {"role": "assistant", "content": "[IN:PAUSE_TIMER  [SL:METHOD_TIMER timer ] ]"},
    {"role": "user", "content": "discontinue timer"}
]

completion = client.chat.completions.create(
    model="qwen3-0.6b",
    messages=messages,
    stream=True,
    temperature=0,     
    top_p=1.0,        
    max_tokens=200,    
    extra_body={
        "enable_thinking": False,
    }
)

reasoning_content = "" 
answer_content = "" 
is_answering = False  
print("=" * 20 + "Thinking Process" + "=" * 20 )

for chunk in completion:
    if not chunk.choices:
        print("Usage:")
        print(chunk.usage)
        continue

    delta = chunk.choices[0].delta

    if hasattr(delta, "reasoning_content") and delta.reasoning_content is not None:
        if not is_answering:
            print(delta.reasoning_content, end="", flush=True)
        reasoning_content += delta.reasoning_content

    # Received content, starting to respond
    if hasattr(delta, "content") and delta.content:
        if not is_answering:
            print("\n" + "=" * 20 + "Complete Response" + "=" * 20)
            is_answering = True
        print(delta.content, end="", flush=True)
        answer_content += delta.content

    if is_answering and hasattr(chunk.choices[0], "finish_reason") and chunk.choices[0].finish_reason is not None:
        print("\n" + "=" * 20 + "Prompt Messages" + "=" * 20)
        # 将历史按 assistant:xxx\n user:xxx\n 格式输出
        for msg in messages:
            if msg["role"] == "assistant":
                print(f"assistant: {msg['content']}")
            elif msg["role"] == "user":
                print(f"user: {msg['content']}")
        print(f"prediction: {answer_content.strip()}")
        print("=" * 55)